In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import sys
sys.path.append('../lib')
import shane
import utils
import mint_v2


# Comparison of Shane's model and MINT-V2

This is a simple re-execution of [Shane's final tokenomics spreadsheet](https://docs.google.com/spreadsheets/d/1-_G8VCQ7WbytNmps_N6LhQJatPN3CvV6z2_e0bN9cC0/edit?usp=sharing) and a side-by-side execution of the MINT-V2 code.

This is just an static test amid to be used only to compare the models against the spreadsheet calculation.

# Parameters

In [2]:
# Download "Performance" card from https://poktscan.com/explore?tab=chains 
CSV_FILE = '../data/chains_performance_2024-05-20_SAMPLE.csv'

# Network Conditions
network_macro = dict()
network_macro['POKT_value'] = 0.50               # USD/POKT
network_macro['total_supply'] = 1679901301.43    # POKT
network_macro['POKT_stake_per_node'] = 60e3      # POKT (irrelevant but for APR calcualtion, which is a no-goal)
network_macro['relay_multiplier'] = 1           # -
network_macro['base_relay_CU_cost'] = 100           # CUs/relay
network_macro['compute_unit_fee'] = 0.0000000085 # USD/CU
network_macro['mint_share'] = {
                                    "DAO" : 0.10,
                                    "Validator" : 0.05,
                                    "Suplier" : 0.7,
                                    "Source" : 0.15,
                                }

assert sum(network_macro['mint_share'].values()) == 1, "The mint share must sum 1 (100%%)"

### Read Chains Data

In [3]:
# Read chains data
chains_df = utils.get_chain_data(CSV_FILE, relay_multiplier=network_macro['relay_multiplier'], spreadsheet_compat = True)
# Add compute cost
chains_df = utils.add_compute_cost(chains_df, sample_type='uniform', mean=network_macro['base_relay_CU_cost'])
# To copy spreadsheet, set the solana custom to twice the base cost
chains_df.loc[chains_df['Chain'] == 'Solana Custom (C006)', 'Compute Cost'] = network_macro['base_relay_CU_cost']*2

In [4]:
# From data
network_macro['daily relays'] = chains_df['Relays'].sum()
network_macro['supplier nodes'] = chains_df['Active Nodes'].sum()

### Shane's Parameters

In [5]:
shane_params = dict()

# Suplier Boost
shane_params['deflation_threshold'] = 25e9      # relays by day
shane_params['max_mint_per_day'] = 180e3       # POKT
# Sources Boost
shane_params['cutoff_point'] = 20e9             # relays/day
shane_params['prop_of_CUF'] = 0.7               # -
# Miscellaneous (Sources Boost)
shane_params['prop_ceiling'] = 0.0              # -
shane_params['floor_relays'] = 5e9              # relays by day
shane_params['gfpr_rar_ceiling_relays'] = 1.5e9 # relays by day
shane_params['gfpr_rar_prop_ceiling'] = 0.15    # -
shane_params['gfpr_rar_floor_relays'] = 5e6     # relays by day
shane_params['gfpr_rar_prop_floor'] = 0.9       # -
shane_params['gfpr_rar_max_supplier_boost'] = 40e3 # USD


### MINT-V2 Parameters

In [6]:
mint_v2_params = deepcopy(mint_v2.data_dict_defaults)

# Ecosystem Costs and Charges
mint_v2_params["MaturityComputeUnitsCharge"]                 = 0.85e-8                     # USD/ComputeUnits
mint_v2_params["MaturityComputeUnitsCost"]                   = 0.85e-8              # USD/ComputeUnits
mint_v2_params["SupplyGrowCap"]                       = 0.025                        # times
# Servicers
mint_v2_params["MaxBootstrapServicerCostPerComputeUnits"]    = 5e-8           # USD/ComputeUnits
mint_v2_params["ServicersBootstrapUnwindStart"]       = 250                           # Billon ComputeUnits/day
mint_v2_params["ServicersBootstrapEnd"]               = 2500                          # Billon ComputeUnits/day
mint_v2_params["MinUsdMint"]               = 20e3                           # USD/day

# Gateways
mint_v2_params["MinBootstrapGatewayFeePerComputeUnits"]      = 0.0000000085                    # USD/ComputeUnits
mint_v2_params["GatewaysBootstrapUnwindStart"]        = 250                           # Billon ComputeUnits/day
mint_v2_params["GatewaysBootstrapEnd"]                = 2500                           # Billon ComputeUnitss/day

# Check that the minimum minting is not higher than the maturity (miny <= burn) phase miniting
assert mint_v2_params["MinUsdMint"] <= mint_v2_params["MaturityComputeUnitsCharge"]*mint_v2_params["GatewaysBootstrapEnd"]*1e9


# Compare

In [7]:
# Process Shane
shanes_per_service_df, shanes_results  = shane.process(chains_df, network_macro, shane_params)
# Process MINT-V2
mint_v2_per_service_df, mint_v2_results  = mint_v2.process(chains_df, network_macro, mint_v2_params)

In [8]:
# Comparison dataframe on the spreadsheet highlighted results (in green)
data_rows = list()

for actor, results in zip(['Shane', 'MINT-V2'], [shanes_results, mint_v2_results]):
    row_values = list()
    row_values.append(actor)
    # Supply growth rate
    row_values.append(((results['Total_Mint'] - results['Total_Burn'])*365.25/network_macro['total_supply'])*100.)
    # Over-minted total
    row_values.append(results['Total_Mint']-results['Total_Burn'])
    # POKT burnt
    row_values.append(results['Total_Burn'])
    # Suplier monthly income in USD
    month_income_usd = results['Total_Mint_base']*network_macro['mint_share']['Suplier']*30.4*network_macro['POKT_value'] /network_macro['supplier nodes']
    row_values.append(month_income_usd)
    # Suplier APR
    row_values.append(((month_income_usd/30.4)*365./(network_macro['POKT_stake_per_node']*network_macro['POKT_value']))*100.)
    # Over-mint per day due to supliers extra minting
    row_values.append(results['Total_Mint_base']-results['Total_Burn'])
    # Over-minted to sources
    row_values.append(results['Total_Mint_others'])
    

    data_rows.append(row_values)

results_df = pd.DataFrame(data_rows, columns=['Model', 
                                              'Supply Growth Rate [%]', 
                                              'Over-minting per Day [POKT]', 
                                              'POKT burnt per Day [POKT]',
                                              'Suplier Income per Day [USD]',
                                              'Suplier APR (USD) [%]',
                                              'Over-mint per Day for Supliers [POKT]',
                                              'Over-mint per Day for Sources [POKT]',
                                              ])


In [9]:
results_df

,Model,Supply Growth Rate [%],Over-minting per Day [POKT],POKT burnt per Day [POKT],Suplier Income per Day [USD],Suplier APR (USD) [%],Over-mint per Day for Supliers [POKT],Over-mint per Day for Sources [POKT]
0,Shane,1.036518,47672.750233,984.486454,2.577186,0.103144,47068.886001,603.864233
1,MINT-V2,0.666231,30642.086477,984.486454,1.696188,0.067885,30642.086477,0.000000
